# Parameter links

This notebook contains a short description how parameter links work in Python modeling packages that support this feature.
The goal is to use this info as a basis for discussion what to put in Gammapy (see https://github.com/gammapy/gammapy/pull/1971).

TODO: check how parameter links affect the optimiser interface, i.e. amend the examples to something where the parameter list seen by the optimiser is clear.

## What is it?

There are different ways to achieve linked parameters in a Python modeling package:

1. Different models refer to the same Python `Parameter` objects. (Using the intrinsic feature that Python "variables" are references to objects, and there can be many references to the same object. This can be used in Gammapy now, although the way our model `__init__` works makes this awkward: it always creates new `Parameter` objects, so one would have to replace the references after `__init__`.
2. Somewhere in the likelihood evaluation there can be code that updates the value of some parameters based on other parameters, i.e. something like `a.value = b.value`. This is very flexible in one sence (one can have `a.value = 3 * b.value ** 2`), but not flexible concerning giving power to the user, they would have to write their own functions and attach it to the likelihood or model evaluation somehow, e.g. via sub-classing. Very hard to teach to users and likely error-prone, not feasible for 100s of gamma-ray astronomers. This could work if the the goal isn't arbitrary linking of parameters by users, but only as a mechanism used internally to have multiple datasets and a joint likelihood, where it's clear what links should be created on `__init__`.
3. Add some mechanism (property, descriptor, method, ...) as a new `link` feature on the `Parameter` class and / or `Model` class (or maybe `Parameters`?). As far as I can tell in all the existing solutions the link is asymmetric, linking parameter `a` to `b` is something different from linking `b` to `a`. Some solutions use a "push/tell/forward/eager" scheme, where parameter value updates propagate. Other solutions use a "pull/ask/observe/lazy" scheme, where parameter values are computed on access. Some solutions are simple, just linking Python objects, others have callables in the end-user API that define the relationship.

So clearly it will be difficult to make this design choice for Gammapy.

## Existing solutions

Let's have a look how linked parameters are done in existing Python modeling packages:

* [sherpa](https://sherpa.readthedocs.io/en/latest/models/index.html#params-link) has a `Parameter.link` attribute, and a `CompoundParameter` class to support expressions for links.
* [astropy](http://docs.astropy.org/en/stable/modeling/fitting.html) has a `Parameter.tied` attribute (one example on that page) which is a funcion.
* [astromodels](https://astromodels.readthedocs.io/en/latest/Model_tutorial.html#linking-parameters) has a [Model.link](https://astromodels.readthedocs.io/en/latest/api/astromodels.core.html#astromodels.core.model.Model.link) method properties to support linking like [Model.linked_parameters](https://astromodels.readthedocs.io/en/latest/api/astromodels.core.html#astromodels.core.model.Model.linked_parameters) and [Parameter.auxiliary_variable](https://astromodels.readthedocs.io/en/latest/api/astromodels.core.html#astromodels.core.parameter.Parameter.auxiliary_variable)
* [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#constrained-minimization-of-multivariate-scalar-functions-minimize) supports constraints for some optimizers.
* [lmfit](https://lmfit.github.io/lmfit-py/constraints.html) has constraints, which I think can also be parameter links.
* [tensorflow](https://www.tensorflow.org/api_docs/python/tf/Variable#constraint) has `Variable.constraint`, not sure how it works.

## Notes

Note that there are two different things:
* linking parameters reduces the number of parameters to optimise. E.g. if there are 10 parameters, and 3 are linked, the optimiser should see 7 parameters.
* constraining parameters limits the domain of allowed values, but keeps the number of parameters the same.

In some modeling framework, only one or the other is supported, in some both. Also, in terms of API, how links or constraints works can be separate, or can be mixed.

In Gammapy we currently support min / max constraints on parameters, but not more general constraints on the domain.

## Examples

Let's have a quick look and try to get some working examples of linked parameters.

### Gammapy

Link parameters in Gammapy by referring to the same `Parameter` object.

In [1]:
from gammapy.spectrum.models import ConstantModel

In [2]:
a = ConstantModel(1)
b = ConstantModel(2)

# Create link
a.parameters.parameters[0] = b.parameters['const']
assert a.parameters['const'] is b.parameters['const']

# The link isn't visible in printout
# This would probably be a problem in debugging
# We could add parameter `id` (using the Python `id` function)
# to the repr to help with that issue
print(a)
print(a.parameters['const'])
print(b)
print(b.parameters['const'])

ConstantModel

Parameters: 

	 name   value   error unit min max frozen
	----- --------- ----- ---- --- --- ------
	const 2.000e+00   nan      nan nan  False
Parameter(name='const', value=2.0, factor=2.0, scale=1.0, unit='', min=nan, max=nan, frozen=False)
ConstantModel

Parameters: 

	 name   value   error unit min max frozen
	----- --------- ----- ---- --- --- ------
	const 2.000e+00   nan      nan nan  False
Parameter(name='const', value=2.0, factor=2.0, scale=1.0, unit='', min=nan, max=nan, frozen=False)


In [3]:
# Check that link is working
# Updating value for one parameter changes the other
# (same for all properties like min / max / unit)
a.parameters['const'].value = 42
assert b.parameters['const'].value == 42

# Note that the link goes both ways
# We can also set the value in `b` and `a` will update
b.parameters['const'].value = 43
assert a.parameters['const'].value == 43

Note how we can only do this after model init,
and also that `Model` or `Parameter` has no API yet,
we have to access the Python list in `model.parameters.parameters`

### Sherpa

An example of linked parameters in Sherpa is [here](https://sherpa.readthedocs.io/en/latest/models/index.html#params-link), but we'll make our own to have something to execute and investigate here.

In [4]:
from sherpa.models import Gauss1D, Const1D, Parameter

In [5]:
a = Const1D('a')
b = Const1D('b')

# Sherpa only allows setting parameter vals after model init:
a.c0 = 1
b.c0 = 2

# Create link from origin `a.c0` to target `b.c0`
a.c0 = b.c0

# The link is visible in the printout
print(a)
print(a.c0)
print(b)
print(b.c0)

a
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   a.c0         linked            2               expr: b.c0           
val         = 2.0
min         = -3.4028234663852886e+38
max         = 3.4028234663852886e+38
units       = 
frozen      = True
link        = b.c0
default_val = 2.0
default_min = -3.4028234663852886e+38
default_max = 3.4028234663852886e+38
b
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   b.c0         thawed            2 -3.40282e+38  3.40282e+38           
val         = 2.0
min         = -3.4028234663852886e+38
max         = 3.4028234663852886e+38
units       = 
frozen      = False
link        = None
default_val = 2.0
default_min = -3.4028234663852886e+38
default_max = 3.4028234663852886e+38


Updating the link target value will update the link origin parameter as expected.
I see [here](https://github.com/sherpa/sherpa/blob/fae4ccb0be524b970138bb81e1e00ddc330288c7/sherpa/models/parameter.py#L150-L174) that this is achieved via the get / set of the value property.

In [6]:
b.c0 = 42
assert a.c0.val == 42
print(a)
print(a.c0)

a
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   a.c0         linked           42               expr: b.c0           
val         = 42.0
min         = -3.4028234663852886e+38
max         = 3.4028234663852886e+38
units       = 
frozen      = True
link        = b.c0
default_val = 42.0
default_min = -3.4028234663852886e+38
default_max = 3.4028234663852886e+38


In [7]:
# Note that `a.c0._val` is still at the old value
# If a parameter has a `link` set, it's own `_val` becomes unused and irrelevant
print(a.c0._val)

1.0


In [8]:
# The other direction doesn't work the same.
# updating the value of the origin `a.c0` will not update the link target `b.c0`,
# instead it seems to set the link to None
a.c0 = b.c0
a.c0 = 43
print(a)
print(b)
print(a.c0)

a
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   a.c0         thawed           43 -3.40282e+38  3.40282e+38           
b
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   b.c0         thawed           42 -3.40282e+38  3.40282e+38           
val         = 43.0
min         = -3.4028234663852886e+38
max         = 3.4028234663852886e+38
units       = 
frozen      = False
link        = None
default_val = 43.0
default_min = -3.4028234663852886e+38
default_max = 3.4028234663852886e+38


In [9]:
# TODO: check how the linking stuff influences the optimiser interface

In [10]:
# Note that Sherpa parameter linking features are more complex / powerful.
# See https://sherpa.readthedocs.io/en/latest/models/index.html#functional-relationship

### Astropy

Following the example from [here](http://docs.astropy.org/en/stable/modeling/fitting.html).

In [11]:
from astropy.modeling.models import Gaussian1D

def tiedfunc(g1):
   return 10 * g1.stddev

g1 = Gaussian1D(amplitude=1, mean=2, stddev=3)
g1.mean.tied = tiedfunc
print(g1.mean)
print(g1.parameters)

Parameter('mean', value=2.0, tied=<function tiedfunc at 0x1a1aa0e1e0>)
[1. 2. 3.]


In [12]:
g1.stddev = 42
g1

<Gaussian1D(amplitude=1., mean=2., stddev=42.)>

In [13]:
g1.amplitude = 43
g1

<Gaussian1D(amplitude=43., mean=2., stddev=42.)>

I don't know how this is supposed to work. There doesn't seem to be any parameter linking kicking in when setting values on either side of the link!?

There is a test [here](https://github.com/astropy/astropy/blob/a671d49147a34145a6e354187ed1a1da8b9a89ee/astropy/modeling/tests/test_constraints.py#L46) that shows parameter linking working in an optimisation example, but I don't know yet how the optimiser interacts with the model or links. To be investigated ...

### astromodels

TODO: add simple example following https://astromodels.readthedocs.io/en/latest/Model_tutorial.html#linking-parameters and show how it works

### lmfit

TODO: add simple example following https://lmfit.github.io/lmfit-py/constraints.html and show how it works